In [62]:
# Imports des bibliothèques
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import metrics
import warnings

warnings.filterwarnings('ignore')

# Imports des données
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")

# Valeurs manquantes
df_train['Age'].fillna(df_train['Age'].median(), inplace=True)
df_test['Age'].fillna(df_train['Age'].median(), inplace=True)
df_train['Embarked'].fillna(df_train['Embarked'].mode()[0], inplace=True)
df_test['Embarked'].fillna(df_train['Embarked'].mode()[0], inplace=True)
df_test['Fare'].fillna(df_train['Fare'].median(), inplace=True)

# Extraction des titres
df_train['Title'] = df_train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df_test['Title'] = df_test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Regrouper les titres
common_titles = ['Mr', 'Miss', 'Mrs', 'Master']
df_train['Title'] = df_train['Title'].apply(lambda x: x if x in common_titles else 'Rare')
df_test['Title'] = df_test['Title'].apply(lambda x: x if x in common_titles else 'Rare')

# Encoder les colonnes catégorielles y compris 'Title'
categorical_columns = ['Sex', 'Embarked', 'Title']
label_encoders = {}
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    df_train[col] = label_encoders[col].fit_transform(df_train[col])
    df_test[col] = label_encoders[col].transform(df_test[col])

# On supprime les colonnes inutiles
columns_to_drop = ['Cabin', 'Ticket', 'Name']
df_train.drop(columns_to_drop, axis=1, inplace=True)
df_test.drop(columns_to_drop, axis=1, inplace=True)

# Inclure le titre dans les caractéristiques
X = df_train.drop('Survived', axis=1)
# 'Title' est déjà inclus dans X car il fait partie de df_train après encodage

# Appliquer la même transformation au test set
df_test_features = df_test.copy()

# Séparation des features et de la target
Y = df_train['Survived']

# On sépare les données en train et validation
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.3, random_state=42)

# Dimensionnement
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
df_test_scaled = scaler.transform(df_test_features)

# Définition de la grille des hyperparamètres
param_grid_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1, 0.2],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5]
}

# Entraînement du modèle avec GridSearchCV
gradient_boosting = GradientBoostingClassifier(random_state=42)
grid_search_gb = GridSearchCV(
    estimator=gradient_boosting,
    param_grid=param_grid_gb,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

# On entraîne le modèle
grid_search_gb.fit(X_train_scaled, Y_train)

# On récupère le meilleur modèle
best_gradient_boosting = grid_search_gb.best_estimator_

# On affiche les meilleurs paramètres
print(f"Meilleurs paramètres: {grid_search_gb.best_params_}")

# Évaluation sur le jeu de validation
Y_val_pred = best_gradient_boosting.predict(X_val_scaled)
f1_score_val = metrics.f1_score(Y_val, Y_val_pred, average='macro')
print(f'Validation Macro F1 Score: {f1_score_val}')

# Prédiction sur le test set
test_predictions = best_gradient_boosting.predict(df_test_scaled)

# Créer un DataFrame avec les prédictions
submission = pd.DataFrame({
    'PassengerId': df_test['PassengerId'],
    'Survived': test_predictions
})

# Exporter en CSV
submission.to_csv('./data/submission.csv', index=False)
print("Fichier de soumission créé avec succès !")


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Meilleurs paramètres: {'learning_rate': 0.2, 'max_depth': 3, 'min_samples_split': 2, 'n_estimators': 100}
Validation Macro F1 Score: 0.8085714285714285
Fichier de soumission créé avec succès !
